## Data Cleaning for the Discrete Voter Model

This notebook cleans demographic and election data for the Discrete Voter Model for:
1. North Carolina, obtained from the [North Carolina State Board of Elections](https://www.ncsbe.gov/Public-Records-Data-Info/Election-Results-Data)
2. Chicago, obtained from [MGGG](https://mggg.org/)

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [4]:
# Import written packages

import make_grid as mg

import dvm
import dvm_plot

import tools
import election

### Chicago

#### Demographics By Precinct

In [10]:
raw_chi_demo_2011 = pd.read_csv('raw_electoral_data/chicago/chicago_demo.csv')

In [11]:
raw_chi_demo_2011

,JOINID,PER_WHITE,PER_BLACK,PER_HISP,PER_ASIAN,PER_OT,MG_VOTES,MG_E,MG_WIL,MG_F,...,MRO_E,MRO_G,DPP_VOTES,DPP_C,DPP_S,Maj Cand?,Plu Cand?,RO Winner?,Change?,GE 2nd Place?
0,C01009,0.325052,0.039337,0.566253,5.175983e-02,1.759834e-02,198,0.439394,0.060606,0.065657,...,0.401515,0.598485,347,0.475504,0.524496,None,Rahm,Chuy,Yes,Chuy
1,C01027,0.397270,0.030172,0.515805,2.801725e-02,2.873563e-02,170,0.435294,0.041176,0.082353,...,0.477733,0.522267,388,0.435567,0.564433,None,Rahm,Chuy,Yes,Chuy
2,C01028,0.094950,0.450505,0.430303,6.060629e-03,1.818182e-02,132,0.484848,0.121212,0.015152,...,0.383459,0.616541,157,0.751592,0.248408,None,Rahm,Chuy,Yes,Chuy
3,C01031,0.289492,0.104384,0.540014,4.105776e-02,2.505219e-02,209,0.473684,0.047847,0.047847,...,0.440860,0.559140,414,0.417874,0.582126,None,Rahm,Chuy,Yes,Chuy
4,C01035,0.549242,0.015152,0.381944,3.156566e-02,2.209596e-02,221,0.443439,0.018100,0.090498,...,0.484950,0.515050,447,0.398210,0.601790,None,Rahm,Chuy,Yes,Chuy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2064,C10036,0.046868,0.426126,0.522244,4.761188e-03,1.280000e-09,78,0.397436,0.166667,0.025641,...,0.356322,0.643678,84,0.678571,0.321429,None,"Tie (Chuy, Rahm)",Chuy,Yes,Wilson
2065,C16007,0.015564,0.612840,0.362516,6.485080e-04,8.430610e-03,166,0.319277,0.265060,0.042169,...,0.454955,0.545045,301,0.614618,0.385382,None,"Tie (Chuy, Rahm)",Chuy,Yes,Wilson
2066,C26038,0.021516,0.125000,0.847336,6.320000e-10,6.147541e-03,102,0.421569,0.127451,0.019608,...,0.309524,0.690476,159,0.528302,0.471698,None,"Tie (Chuy, Rahm)",Chuy,Yes,Wilson
2067,C49006,0.499615,0.294526,0.113338,5.319968e-02,3.932151e-02,225,0.435556,0.066667,0.040000,...,0.462121,0.537879,395,0.410127,0.589873,None,"Tie (Chuy, Rahm)",Chuy,Yes,Wilson


In [12]:
raw_chi_demo_2011.columns

Index(['JOINID', 'PER_WHITE', 'PER_BLACK', 'PER_HISP', 'PER_ASIAN', 'PER_OT',
       'MG_VOTES', 'MG_E', 'MG_WIL', 'MG_F', 'MG_G', 'MG_WAL', 'MRO_VOTES',
       'MRO_E', 'MRO_G', 'DPP_VOTES', 'DPP_C', 'DPP_S', 'Maj Cand?',
       'Plu Cand?', 'RO Winner?', 'Change?', 'GE 2nd Place?'],
      dtype='object')

In [54]:
raw_chi_demo_2011['pop'] = raw_chi_demo_2011['MG_VOTES'] + raw_chi_demo_2011['DPP_VOTES']

In [55]:
raw_cols_2011 = ['JOINID', 'PER_WHITE', 'PER_BLACK', 'PER_HISP', 'PER_ASIAN', 'PER_OT', 'pop']

In [56]:
chi_demo_2011_v2 = raw_chi_demo_2011[raw_cols_2011]

In [57]:
chi_demo_2011_v2['white'] = (chi_demo_2011_v2['PER_WHITE'] * chi_demo_2011_v2['pop']).astype(int)
chi_demo_2011_v2['hispanic'] = (chi_demo_2011_v2['PER_HISP'] * chi_demo_2011_v2['pop']).astype(int)
chi_demo_2011_v2['asian'] = (chi_demo_2011_v2['PER_ASIAN'] * chi_demo_2011_v2['pop']).astype(int)
chi_demo_2011_v2['black'] = (chi_demo_2011_v2['PER_BLACK'] * chi_demo_2011_v2['pop']).astype(int)
chi_demo_2011_v2['other'] = (chi_demo_2011_v2['PER_OT'] * chi_demo_2011_v2['pop']).astype(int)

/Users/hakeemangulu/.local/share/virtualenvs/code-9G5VMzgS/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/hakeemangulu/.local/share/virtualenvs/code-9G5VMzgS/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/hakeemangulu/.local/share/virtualenvs/code-9G5VMzgS/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is

In [58]:
chi_demo_2011_v3 = chi_demo_2011_v2.rename(columns = {'JOINID':'prec_id'})

In [59]:
cols_2011 = ['prec_id', 'black', 'hispanic', 'asian', 'white', 'other']

In [60]:
chi_demo_2011_v3

,prec_id,PER_WHITE,PER_BLACK,PER_HISP,PER_ASIAN,PER_OT,pop,white,hispanic,asian,black,other
0,C01009,0.325052,0.039337,0.566253,5.175983e-02,1.759834e-02,545,177,308,28,21,9
1,C01027,0.397270,0.030172,0.515805,2.801725e-02,2.873563e-02,558,221,287,15,16,16
2,C01028,0.094950,0.450505,0.430303,6.060629e-03,1.818182e-02,289,27,124,1,130,5
3,C01031,0.289492,0.104384,0.540014,4.105776e-02,2.505219e-02,623,180,336,25,65,15
4,C01035,0.549242,0.015152,0.381944,3.156566e-02,2.209596e-02,668,366,255,21,10,14
...,...,...,...,...,...,...,...,...,...,...,...,...
2064,C10036,0.046868,0.426126,0.522244,4.761188e-03,1.280000e-09,162,7,84,0,69,0
2065,C16007,0.015564,0.612840,0.362516,6.485080e-04,8.430610e-03,467,7,169,0,286,3
2066,C26038,0.021516,0.125000,0.847336,6.320000e-10,6.147541e-03,261,5,221,0,32,1
2067,C49006,0.499615,0.294526,0.113338,5.319968e-02,3.932151e-02,620,309,70,32,182,24


In [61]:
final_chi_demo_2011 = chi_demo_2011_v3[cols_2011]

In [62]:
final_chi_demo_2011

,prec_id,black,hispanic,asian,white,other
0,C01009,21,308,28,177,9
1,C01027,16,287,15,221,16
2,C01028,130,124,1,27,5
3,C01031,65,336,25,180,15
4,C01035,10,255,21,366,14
...,...,...,...,...,...,...
2064,C10036,69,84,0,7,0
2065,C16007,286,169,0,7,3
2066,C26038,32,221,0,5,1
2067,C49006,182,70,32,309,24


In [64]:
final_chi_demo_2011.to_csv('electoral_data/chi_demo.csv', index=False)

#### City Council

#### 2011

In [135]:
raw_chi_cc_2011 = pd.read_csv('raw_electoral_data/chicago/chicago_cc_2011.csv')

In [136]:
raw_chi_cc_2011

,Ward,Precinct,PROCO ''JOE'' MORENO,RONALD J. BALTIERRA,DEBORAH LOPEZ,GENITA C. ROBINSON,BOB FIORETTI,FEDERICO SCIAMMARELLA,JAMES A. BOSCO,MELISSA CALLAHAN,...,STEVEN CHERESKA,JOSE E. ARTEAGA,PHILIP R. BERNSTEIN,JOE MOORE,BRIAN WHITE,BERNARD L. STONE,MICHAEL CHARLES MOSES,DEBRA L. SILVERSTEIN,GREG BREWER,AHMED KHAN
0,1,1,187.0,9.0,54.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,140.0,12.0,58.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,3,122.0,11.0,18.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,4,153.0,13.0,27.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,5,135.0,17.0,30.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2565,50,41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,127.0,28.0,132.0,78.0,3.0
2566,50,42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,54.0,18.0,76.0,96.0,2.0
2567,50,43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,73.0,11.0,80.0,75.0,52.0
2568,50,44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,140.0,7.0,88.0,11.0,8.0


In [137]:
raw_chi_cc_2011['prec_id'] = 'C' + raw_chi_cc_2011['Ward'].map('{:02d}'.format) + '0' + raw_chi_cc_2011['Precinct'].map('{:02d}'.format)

In [162]:
"test ".strip()

'test'

In [167]:
cc_2011_dfs = {}
for ward in raw_chi_cc_2011['Ward'].unique():
    ward_df = raw_chi_cc_2011[raw_chi_cc_2011['Ward'] == ward]
    
    ward_df.columns = [col.title().strip() for col in raw_chi_cc_2011.columns]
    ward_df.rename(columns = {'Prec_Id': 'prec_id'}, inplace=True)
    
    ward_df.dropna(axis=1, inplace=True)
    ward_df.drop(['Ward', 'Precinct'], axis=1, inplace=True)
    
    for col in ward_df.columns:
        if col == 'prec_id':
            continue
        ward_df[col] = ward_df[col].astype(int)
    
    cc_2011_dfs[ward] = ward_df

/Users/hakeemangulu/.local/share/virtualenvs/code-9G5VMzgS/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/Users/hakeemangulu/.local/share/virtualenvs/code-9G5VMzgS/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/hakeemangulu/.local/share/virtualenvs/code-9G5VMzgS/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

In [169]:
cc_2011_dfs[1]

,Proco ''Joe'' Moreno,Ronald J. Baltierra,Deborah Lopez,prec_id
0,187,9,54,C01001
1,140,12,58,C01002
2,122,11,18,C01003
3,153,13,27,C01004
4,135,17,30,C01005
5,151,12,53,C01006
6,170,7,50,C01007
7,166,14,44,C01008
8,172,6,37,C01009
9,159,29,42,C01010


In [139]:
for ward, df in cc_2011_dfs.items():
    df.to_csv(f"electoral_data/chicago_cc_2011/chi_cc_2011_ward{ward}.csv", index=False)

#### 2015

In [104]:
raw_chi_cc_2015 = pd.read_csv('raw_electoral_data/chicago/chicago_cc_2015.csv')

In [105]:
raw_chi_cc_2015

,Ward/Precinct ID,PROCO ''JOE'' MORENO,ANNE SHAW,ANDREW HAMILTON,RONDA LOCKE,STEPHEN NIKETOPOULOS,BITA BUENROSTRO,BRIAN HOPKINS,ALYX S. PATTISON,CORNELL WILSON,...,AMY CRAWFORD,DENICE L. DAVIS,AMEYA PAWAR,RORY A. FIEDLER,HARRY OSTERMAN,JOE MOORE,DON GORDON,DEBRA L. SILVERSTEIN,SHAJAN M. KURIAKOSE,ZEHRA QUADRI
0,101,104.0,46.0,4.0,33.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,102,83.0,52.0,15.0,30.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,103,80.0,56.0,3.0,69.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,104,71.0,44.0,6.0,104.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,105,85.0,13.0,2.0,66.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2064,5036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,165.0,41.0,30.0
2065,5037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,151.0,30.0,35.0
2066,5038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.0,33.0,12.0
2067,5039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.0,28.0,20.0


In [122]:
(raw_chi_cc_2015['Ward/Precinct ID']).astype(str).str.strip().str[:-2]

0        1
1        1
2        1
3        1
4        1
        ..
2064    50
2065    50
2066    50
2067    50
2068    50
Name: Ward/Precinct ID, Length: 2069, dtype: object

In [121]:
raw_chi_cc_2015['Precinct'] = (raw_chi_cc_2015['Ward/Precinct ID']).astype(str).str.strip().str[-2:]

In [123]:
raw_chi_cc_2015['Ward'] = (raw_chi_cc_2015['Ward/Precinct ID']).astype(str).str.strip().str[:-2]

In [132]:
raw_chi_cc_2015['Ward'] = (raw_chi_cc_2015['Ward']).astype(int).map('{:02d}'.format)

In [133]:
raw_chi_cc_2015['prec_id'] = 'C' + raw_chi_cc_2015['Precinct'] + '0' + raw_chi_cc_2015['Ward']

In [141]:
raw_chi_cc_2015.columns

Index(['Ward/Precinct ID', 'PROCO ''JOE'' MORENO', 'ANNE SHAW',
       'ANDREW HAMILTON', 'RONDA LOCKE', 'STEPHEN NIKETOPOULOS',
       'BITA BUENROSTRO', 'BRIAN HOPKINS', 'ALYX S. PATTISON',
       'CORNELL WILSON',
       ...
       'RORY A. FIEDLER', 'HARRY OSTERMAN', 'JOE MOORE', 'DON GORDON',
       'DEBRA L. SILVERSTEIN', 'SHAJAN M. KURIAKOSE', 'ZEHRA QUADRI',
       'Precinct', 'Ward', 'prec_id'],
      dtype='object', length=189)

In [173]:
cc_2015_dfs = {}
for ward in raw_chi_cc_2015['Ward'].unique():
    ward_df = raw_chi_cc_2015[raw_chi_cc_2015['Ward'] == ward]
    
    ward_df.columns = [col.title() for col in raw_chi_cc_2015.columns]
    ward_df.rename(columns = {'Prec_Id': 'prec_id'}, inplace=True)
    
    ward_df.dropna(axis=1, inplace=True)
    ward_df.drop(['Ward', 'Precinct', 'Ward/Precinct Id'], axis=1, inplace=True)
    
    for col in ward_df.columns:
        if col == 'prec_id':
            continue
        ward_df[col] = ward_df[col].astype(int)
    
    cc_2015_dfs[int(ward)] = ward_df

/Users/hakeemangulu/.local/share/virtualenvs/code-9G5VMzgS/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/Users/hakeemangulu/.local/share/virtualenvs/code-9G5VMzgS/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/hakeemangulu/.local/share/virtualenvs/code-9G5VMzgS/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

In [176]:
cc_2015_dfs[1]

,Proco ''Joe'' Moreno,Anne Shaw,Andrew Hamilton,Ronda Locke,prec_id
0,104,46,4,33,C01001
1,83,52,15,30,C02001
2,80,56,3,69,C03001
3,71,44,6,104,C04001
4,85,13,2,66,C05001
5,103,60,13,31,C06001
6,77,45,3,28,C07001
7,149,32,7,25,C08001
8,136,39,5,16,C09001
9,89,37,7,52,C10001


In [177]:
for ward, df in cc_2015_dfs.items():
    df.to_csv(f"electoral_data/chicago_cc_2015/chi_cc_2015_ward{ward}.csv", index=False)

#### Mayoral

#### 2011

In [196]:
raw_chi_mayor_2011 = pd.read_csv('raw_electoral_data/chicago/chicago_mayor_2011.csv')

In [197]:
raw_chi_mayor_2011

,WARD,PRECINCT,JOINID,Emanuel_M11,DelValle_M11,Braun_M11,Chico_M11,Watkins_M11,Walls_M11
0,1,1,C01001,123,53,3,82,2,1
1,1,2,C01002,149,49,0,34,0,1
2,1,3,C01003,98,31,1,33,0,0
3,1,4,C01004,125,35,3,47,1,0
4,1,5,C01005,133,40,5,23,0,0
...,...,...,...,...,...,...,...,...,...
2565,50,41,C50041,222,26,4,112,1,1
2566,50,42,C50042,150,43,3,46,2,3
2567,50,43,C50043,154,36,25,67,3,2
2568,50,44,C50044,161,9,2,80,0,0


In [198]:
chi_mayor_2011 = raw_chi_mayor_2011.drop(['WARD', 'PRECINCT'], axis=1)

In [199]:
chi_mayor_2011_cols = [col.replace('_M11', '').replace('JOINID', 'prec_id') for col in chi_mayor_2011.columns]
chi_mayor_2011.columns = chi_mayor_2011_cols

In [200]:
chi_mayor_2011

,prec_id,Emanuel,DelValle,Braun,Chico,Watkins,Walls
0,C01001,123,53,3,82,2,1
1,C01002,149,49,0,34,0,1
2,C01003,98,31,1,33,0,0
3,C01004,125,35,3,47,1,0
4,C01005,133,40,5,23,0,0
...,...,...,...,...,...,...,...
2565,C50041,222,26,4,112,1,1
2566,C50042,150,43,3,46,2,3
2567,C50043,154,36,25,67,3,2
2568,C50044,161,9,2,80,0,0


In [201]:
chi_mayor_2011.to_csv(f"electoral_data/chi_mayor_2011.csv", index=False)

#### 2015

In [202]:
raw_chi_mayor_2015 = pd.read_csv('raw_electoral_data/chicago/chicago_mayor_2015.csv')

In [203]:
raw_chi_mayor_2015

,WARD,PRECINCT,JOINID,Emanuel_M15,Wilson_M15,Fioretti_M15,Garcia_M15,Walls_M15
0,10,1,C10001,108,42,3,80,7
1,10,2,C10002,139,9,57,113,2
2,10,3,C10003,144,9,38,110,3
3,10,4,C10004,70,17,9,90,5
4,10,5,C10005,118,10,34,89,2
...,...,...,...,...,...,...,...,...
2064,9,47,C09047,142,69,16,76,15
2065,9,48,C09048,110,68,12,65,22
2066,9,49,C09049,96,54,12,52,20
2067,9,50,C09050,134,61,22,113,31


In [204]:
chi_mayor_2015 = raw_chi_mayor_2015.drop(['WARD', 'PRECINCT'], axis=1)

In [205]:
chi_mayor_2015_cols = [col.replace('_M15', '').replace('JOINID', 'prec_id') for col in chi_mayor_2015.columns]
chi_mayor_2015.columns = chi_mayor_2015_cols

In [206]:
chi_mayor_2015

,prec_id,Emanuel,Wilson,Fioretti,Garcia,Walls
0,C10001,108,42,3,80,7
1,C10002,139,9,57,113,2
2,C10003,144,9,38,110,3
3,C10004,70,17,9,90,5
4,C10005,118,10,34,89,2
...,...,...,...,...,...,...
2064,C09047,142,69,16,76,15
2065,C09048,110,68,12,65,22
2066,C09049,96,54,12,52,20
2067,C09050,134,61,22,113,31


In [207]:
chi_mayor_2015.to_csv(f"electoral_data/chi_mayor_2015.csv", index=False)

#### 2019

In [208]:
raw_chi_mayor_2019 = pd.read_csv('raw_electoral_data/chicago/chicago_mayor_2019.csv')

In [209]:
raw_chi_mayor_2019

,Precinct_Name,WARD,PRECINCT,JOINID,TOTVOTES,JOYCE,VALLAS,WILSON,PRECKWINKLE,DALEY,...,TOTPOP,NH_WHITE,NH_BLACK,NH_ASIAN,HISP,PER_WHITE,PER_BLACK,PER_ASIAN,PER_HISP,PER_NW
0,Ward 01 Precinct 01,1,1,C01001,330,10,8,14,53,34,...,1410.000058,373.000020,111.999998,18.000003,895.000033,0.264539,0.079433,0.012766,0.634752,0.735461
1,Ward 01 Precinct 02,1,2,C01002,248,10,8,19,39,29,...,1713.000082,200.999998,249.000046,9.999997,1227.000041,0.117338,0.145359,0.005838,0.716287,0.882662
2,Ward 01 Precinct 03,1,3,C01003,312,13,11,6,48,44,...,1494.000018,945.999992,30.000001,55.000003,437.000023,0.633199,0.020080,0.036814,0.292503,0.366801
3,Ward 01 Precinct 04,1,4,C01004,307,13,22,5,53,49,...,1449.000038,790.000026,26.000001,51.000003,558.000008,0.545204,0.017943,0.035197,0.385093,0.454796
4,Ward 01 Precinct 05,1,5,C01005,286,10,18,5,45,35,...,1258.999949,953.999958,75.999999,48.999997,146.999996,0.757744,0.060365,0.038920,0.116759,0.242256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2064,Ward 50 Precinct 36,50,36,C50036,290,29,20,8,52,44,...,1701.000001,792.999999,134.999998,367.999999,352.000008,0.466196,0.079365,0.216343,0.206937,0.533804
2065,Ward 50 Precinct 37,50,37,C50037,271,22,19,10,40,32,...,1350.000078,599.000053,160.000000,184.000000,370.000023,0.443704,0.118519,0.136296,0.274074,0.556296
2066,Ward 50 Precinct 38,50,38,C50038,154,18,13,7,23,33,...,663.709993,588.371239,5.593196,43.810053,21.060947,0.886488,0.008427,0.066008,0.031732,0.113512
2067,Ward 50 Precinct 39,50,39,C50039,155,8,20,6,18,40,...,1388.995723,676.997486,78.999993,427.998523,155.999908,0.487401,0.056876,0.308135,0.112311,0.512599


In [211]:
raw_chi_mayor_2019.columns

Index(['Precinct_Name', 'WARD', 'PRECINCT', 'JOINID', 'TOTVOTES', 'JOYCE',
       'VALLAS', 'WILSON', 'PRECKWINKLE', 'DALEY', 'MCCARTHY', 'CHICO',
       'MENDOZA', 'ENYIA', 'FORD', 'SALESGRIFFIN', 'LIGHTFOOT', 'FIORETTI',
       'KOZLAR', 'P_JOYCE', 'P_VALLAS', 'P_WISLON', 'P_PRECKWINKLE', 'P_DALEY',
       'P_MCCARTHY', 'P_CHICO', 'P_MENDOZA', 'P_ENYIA', 'P_FORD',
       'P_SALESGRIFFIN', 'P_LIGHTFOOT', 'P_FIORETTI', 'P_KOZLAR', 'TOTPOP',
       'NH_WHITE', 'NH_BLACK', 'NH_ASIAN', 'HISP', 'PER_WHITE', 'PER_BLACK',
       'PER_ASIAN', 'PER_HISP', 'PER_NW'],
      dtype='object')

In [226]:
raw_chi_mayor_2019_cols = ['JOINID', 'JOYCE', 'VALLAS', 'WILSON', 'PRECKWINKLE', 
                       'DALEY', 'MCCARTHY', 'CHICO', 'MENDOZA', 'ENYIA', 'FORD', 'SALESGRIFFIN', 
                       'LIGHTFOOT', 'FIORETTI', 'KOZLAR']
chi_mayor_2019_cols = [col.title().replace('Joinid', 'prec_id') for col in raw_chi_mayor_2019_cols]

In [229]:
chi_mayor_2019 = raw_chi_mayor_2019[raw_chi_mayor_2019_cols]
chi_mayor_2019.columns = chi_mayor_2019_cols

In [230]:
chi_mayor_2019

,prec_id,Joyce,Vallas,Wilson,Preckwinkle,Daley,Mccarthy,Chico,Mendoza,Enyia,Ford,Salesgriffin,Lightfoot,Fioretti,Kozlar
0,C01001,10,8,14,53,34,3,19,33,59,2,1,90,1,1
1,C01002,10,8,19,39,29,2,14,29,36,2,1,56,2,0
2,C01003,13,11,6,48,44,5,11,23,51,0,0,92,1,5
3,C01004,13,22,5,53,49,9,13,31,32,2,1,69,4,4
4,C01005,10,18,5,45,35,5,9,21,54,0,1,77,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2064,C50036,29,20,8,52,44,7,23,19,21,0,2,61,2,1
2065,C50037,22,19,10,40,32,7,22,25,15,1,2,70,5,0
2066,C50038,18,13,7,23,33,5,17,13,3,0,0,20,0,0
2067,C50039,8,20,6,18,40,13,6,9,10,0,0,14,5,0


In [231]:
chi_mayor_2019.to_csv(f"electoral_data/chi_mayor_2019.csv", index=False)